<a href="https://colab.research.google.com/github/david4129/Global_prof/blob/main/Global_Profitability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.setrecursionlimit(10000)

In [1]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [2]:
#import libraries
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as px1
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
from tabulate import tabulate
pn.extension('plotly')

In [3]:
#Read in dataset
df = pd.read_excel('drive/MyDrive/GLOBAL SUPERSTORE.xlsx')
df.head()

,Row ID,Order ID,Order Date,Delivery Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
#Get information  about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Delivery Date  9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [5]:
#Create a new feature that contains the year for the order date
df['Order Year'] = df['Order Date'].dt.year
df['Order Year'].value_counts()

2017    3312
2016    2587
2015    2102
2014    1993
Name: Order Year, dtype: int64

In [6]:
#Create a list of regions
region_list = list(df['Region'].value_counts().index)
region_list.sort()
#Create a radio button widget
radio_reg = pn.widgets.RadioButtonGroup(name = 'Region', options = region_list, button_type = 'success', value = 'South')
radio_reg

RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')

In [7]:
#Create a reactive doughnut chart
@pn.depends(radio_reg)
def doughnut(region):
    sales = df[df['Region'] == region]
    sales_df = sales.groupby('Category').sum()
    labels = list(sales_df.index)
    values = list(sales_df['Profit'])
    fig4 = px.pie(labels, values = values, hole = 0.4, names = labels, color = labels, width = 400, height = 400)
    fig4.update_layout(title = 'Profit By Category', legend = dict(
    x = 1, y = 0.45), margin = dict(t = 0, b = 0, l = 0, r = 0), title_y = 0.9)
    return fig4
pn.Column(radio_reg, doughnut)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=Plotly)

In [8]:
#Create a reactive multi bar chart
@pn.depends(radio_reg)
def mult_bar(region):
    region_df = df[df['Region'] == region]
    region_group = region_df.groupby('Order Year').sum()
    fig = px1.Figure(data = [px1.Bar(name = 'Sum of Sales', x = region_group.index,
                                 y = region_group['Sales']),
                           px1.Bar(name = 'Sum of Profit', x = region_group.index,
                                 y = region_group['Profit'])])
    fig.update_layout(title = 'Sales and Profit Comparison', width = 400, height = 400)
    return fig
pn.Column(radio_reg, mult_bar)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=Plotly)

In [9]:
#Create a reactive doughnut chart
@pn.depends(radio_reg)
def doughnut2(region):
    sales = df[df['Region'] == region]
    sales_df = sales.groupby('Segment').sum()
    labels = list(sales_df.index)
    values = list(sales_df['Profit'])
    fig4 = px.pie(labels, values = values, hole = 0.4, names = labels, color = labels, width = 400, height = 400)
    fig4.update_layout(title = 'Profit By Segment', legend = dict(
    x = 1, y = 0.45), margin = dict(t = 0, b = 0, l = 0, r = 0), title_y = 0.9)
    return fig4
pn.Column(radio_reg, doughnut2)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=Plotly)

In [10]:
#Create a reactive area chart
@pn.depends(radio_reg)
def area(region):
  df_reg = df[df['Region'] == region]
  sum_year = df_reg.groupby('Order Year').sum()
  sum_year.reset_index(inplace = True)
  fig = px.area(sum_year, x ='Order Year', y = 'Profit',
              hover_data=['Sales'])
  fig.update_xaxes(tickmode='linear', dtick=1)
  fig.update_layout(title = 'Profit By Year', width = 400, height = 400)
  return fig

pn.Column(radio_reg, area)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=Plotly)

In [11]:
#Create a reactive bar chart
@pn.depends(radio_reg)
def bar(region):
  df_reg = df[df['Region'] == region]
  sum_ship = df_reg.groupby('Ship Mode').sum()
  sum_ship.reset_index(inplace = True)
  labels = list(sum_ship['Ship Mode'])
  values = list(sum_ship['Profit'])
  fig = px.bar(x = values, y = labels, orientation='h')
  fig.update_layout(title = 'Profit By Ship Mode', width = 400, height = 400, yaxis_title=None, xaxis_title=None)
  return fig

pn.Column(radio_reg, bar)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=Plotly)

In [12]:
#Create a reactive scatter chart
@pn.depends(radio_reg)
def scatter(region):
  df_reg = df[df['Region'] == region]
  fig = px.scatter(df_reg, x = 'Profit', y = 'Discount', size = 'Sales')
  fig.update_layout(title = 'Profit vs Discount', width = 400, height = 400)
  return fig

pn.Column(radio_reg, scatter)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=Plotly)

In [13]:
#Create a reactive table which outputs top 5 cities by profit in region specified
@pn.depends(radio_reg.param.value)
def top_city(region):
  reg_df = df[df['Region'] == region]
  cust = reg_df.groupby('City').sum()['Profit'].nlargest(5)
  new_cust = cust.reset_index()
  new_cust['Profit'] = np.round(new_cust['Profit'], 1)
  new_cust['Profit'] = new_cust['Profit'].apply(lambda x: '${}'.format(x))
  return new_cust

@pn.depends(radio_reg)
def upd_top_city(new_region):
   new_top_city = top_city(new_region)
   return new_top_city

pn.Column(radio_reg, upd_top_city)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=DataFrame)

In [14]:
#Create a reactive table which outputs top 5 customers by profit in region specified
@pn.depends(radio_reg.param.value)
def aut_cust(region):
  reg_df = df[df['Region'] == region]
  cust = reg_df.groupby('Customer Name').sum()['Profit'].nlargest(5)
  new_cust = cust.reset_index()
  new_cust['Profit'] = np.round(new_cust['Profit'], 1)
  new_cust['Profit'] = new_cust['Profit'].apply(lambda x: '${}'.format(x))
  return new_cust

@pn.depends(radio_reg)
def upd_cust(new_region):
  region = aut_cust(new_region)
  return region

pn.Column(radio_reg, upd_cust)

Column
    [0] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [1] ParamFunction(function, _pane=DataFrame)

In [15]:
#Create header box layout
title = '# Global Store Profitability Dashboard'
wid_text = 'Choose region here'
descr = '## Top 5 most profitable customers'
descr2 = '## Top 5 most profitable cities'
header_box = pn.WidgetBox(title, wid_text, radio_reg, descr, upd_cust, descr2, upd_top_city,
                          width=350, height=800, align="center")
header_box

WidgetBox(align='center', height=800, sizing_mode='fixed', width=350)
    [0] Markdown(str)
    [1] Markdown(str)
    [2] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
    [3] Markdown(str)
    [4] ParamFunction(function, _pane=DataFrame)
    [5] Markdown(str)
    [6] ParamFunction(function, _pane=DataFrame)

In [16]:
#Create main plot
plots_box =  pn.WidgetBox(pn.Column(pn.Row(doughnut, mult_bar, bar), pn.Row(doughnut2, area, scatter)), 
                          height = 800, width=800, sizing_mode="stretch_width")
plots_box

WidgetBox(height=800, sizing_mode='stretch_width', width=800)
    [0] Column
        [0] Row
            [0] ParamFunction(function, _pane=Plotly)
            [1] ParamFunction(function, _pane=Plotly)
            [2] ParamFunction(function, _pane=Plotly)
        [1] Row
            [0] ParamFunction(function, _pane=Plotly)
            [1] ParamFunction(function, _pane=Plotly)
            [2] ParamFunction(function, _pane=Plotly)

In [17]:
dashboard = pn.Row(header_box, plots_box, sizing_mode="stretch_width")
dashboard

Row(sizing_mode='stretch_width')
    [0] WidgetBox(align='center', height=800, sizing_mode='fixed', width=350)
        [0] Markdown(str)
        [1] Markdown(str)
        [2] RadioButtonGroup(button_type='success', name='Region', options=['Central', 'East', ...], value='South')
        [3] Markdown(str)
        [4] ParamFunction(function, _pane=DataFrame)
        [5] Markdown(str)
        [6] ParamFunction(function, _pane=DataFrame)
    [1] WidgetBox(height=800, sizing_mode='stretch_width', width=800)
        [0] Column
            [0] Row
                [0] ParamFunction(function, _pane=Plotly)
                [1] ParamFunction(function, _pane=Plotly)
                [2] ParamFunction(function, _pane=Plotly)
            [1] Row
                [0] ParamFunction(function, _pane=Plotly)
                [1] ParamFunction(function, _pane=Plotly)
                [2] ParamFunction(function, _pane=Plotly)

In [18]:
dashboard.save("my_dashboard1.html", embed=True)